In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf



In [ ]:
! pip install datasets

In [ ]:
from datasets import load_dataset
ds = load_dataset('wider_face',split='train')

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

df=ds.to_pandas()
print(df.index)
print(df.faces.iloc[3]['bbox'])
path=df.image.iloc[3]['path']
def load_image(image_path):
    with open(image_path, 'rb') as f:
        img = Image.open(f)
        return np.array(img)
plt.imshow(load_image(path))

In [ ]:
import cv2
path=df.image.iloc[0]['path']
img = cv2.imread(path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print(img.shape)
for bbox in df.faces.iloc[0]['bbox']:

  x1, y1, x2, y2 = bbox
  cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
plt.imshow(img)

In [ ]:
import pandas as pd
import csv
from sklearn import preprocessing
import cv2

In [ ]:
labels=['face']
le = preprocessing.LabelEncoder()
le.fit(labels)
print(le.classes_)

In [ ]:
def coco_to_yolo(coco_boxes, img_width, img_height,le):
    yolo_boxes = []
    labelcode = le.transform(['face'])[0]
    for box in coco_boxes:
        x_min, y_min, width, height = box
        x_center = x_min + width / 2.0
        y_center = y_min + height / 2.0
        x_center_norm = x_center / img_width
        y_center_norm = y_center / img_height
        width_norm = width / img_width
        height_norm = height / img_height
        yolo_boxes.append([labelcode,x_center_norm, y_center_norm, width_norm, height_norm])
    return yolo_boxes

In [ ]:
import shutil

folder_path = '/kaggle/working/yolov7/split'
shutil.rmtree(folder_path)

In [ ]:
import os
os.makedirs(os.path.join('images'), exist_ok=True)

In [ ]:
def savedatasetfhs(row):
  a=row.name
  if(a< 6500):
    path=row[0]['path']
    img=cv2.imread(path)
    image=tf.keras.preprocessing.image.array_to_img(load_image(row[0]['path']))
    image.save('/kaggle/working/images/image'+str(a)+'.jpeg')
    np.savetxt('/kaggle/working/images/image'+str(a)+'.txt', coco_to_yolo(row[1]['bbox'],img.shape[1],img.shape[0],le))
    #np.save('/content/images/image'+str(a)+'.npy', coco_to_yolo(row[1]['bbox'],img.shape[1],img.shape[0]))
df.apply(savedatasetfhs,axis="columns")

In [4]:
%%capture

!git clone https://github.com/WongKinYiu/yolov7 # Downloading YOLOv7 repository and installing requirements
%cd yolov7
!pip3 install -qr requirements.txt

In [ ]:
!wget "https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt"

In [5]:
!wandb disabled

W&B disabled.


In [ ]:
import os
from sklearn.model_selection import train_test_split

def split_data(folder_path, train_size=0.8):
    # Create the output directories
    os.makedirs(os.path.join('split', 'images', 'train'), exist_ok=True)
    os.makedirs(os.path.join('split', 'images', 'val'), exist_ok=True)
    os.makedirs(os.path.join('split', 'labels', 'train'), exist_ok=True)
    os.makedirs(os.path.join('split', 'labels', 'val'), exist_ok=True)

    # Get the list of image and label files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpeg')]
    label_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

    # Sort the lists of files

    image_files.sort()
    label_files.sort()

    # Split the data into training and validation sets
    train_images, val_images, train_labels, val_labels = train_test_split(image_files, label_files, train_size=train_size)

    # Copy the training images and labels to the output directories
    for image_file, label_file in zip(train_images, train_labels):
        shutil.copy(os.path.join(folder_path, image_file), os.path.join('split', 'images', 'train', image_file))
        shutil.copy(os.path.join(folder_path, label_file), os.path.join('split', 'labels', 'train', label_file))

    # Copy the validation images and labels to the output directories
    for image_file, label_file in zip(val_images, val_labels):
        shutil.copy(os.path.join(folder_path, image_file), os.path.join('split', 'images', 'val', image_file))
        shutil.copy(os.path.join(folder_path, label_file), os.path.join('split', 'labels', 'val', label_file))


folder_path = '/kaggle/working/images'
split_data(folder_path)





In [ ]:
train_img_path = "split/images/train"
train_label_path = "split/labels/train"

val_img_path = "split/images/val"
val_label_path = "split/labels/val"
image_files_train=[f for f in os.listdir(train_img_path) if f.endswith('.jpeg')]
label_files_train=[f for f in os.listdir(train_label_path) if f.endswith('.txt')]
image_files_val=[f for f in os.listdir(val_img_path) if f.endswith('.jpeg')]
label_files_val=[f for f in os.listdir(val_label_path) if f.endswith('.txt')]
image_files_train.sort()
label_files_train.sort()
image_files_val.sort()
label_files_val.sort()
for f in range(len(image_files_train)):
  image_files_train[f]=f'{image_files_train[f].split(".")[0]}'
for f in range(len(label_files_train)):
  label_files_train[f]=f'{label_files_train[f].split(".")[0]}'
for f in range(len(image_files_val)):
  image_files_val[f]=f'{image_files_val[f].split(".")[0]}'
for f in range(len(label_files_val)):
  label_files_val[f]=f'{label_files_val[f].split(".")[0]}'
if(len(image_files_train)==len(label_files_train) and image_files_train== label_files_train):
  print('sucessful train split ')
  print('number of train images: '+str(len(image_files_train)))
  print('number of train labels: '+str(len(label_files_train)))
else:
  print('Unsucessful train split')
if(len(image_files_val)==len(label_files_val) and image_files_val== label_files_val):
  print('sucessful val split ')
  print('number of val images: '+str(len(image_files_val)))
  print('number of val labels: '+str(len(label_files_val)))
else:
  print('Unsucessful val split')

In [ ]:
# creaing .yaml file for specifying about the paths, number of classes, and class names
!echo -e "train: /kaggle/working/yolov7/split/images/train\nval: /kaggle/working/yolov7/split/images/val\n\nnc: 1\nnames: ['face']" >> rbcdet.yaml
!cat 'rbcdet.yaml'

In [ ]:
shutil.copyfile('/kaggle/working/yolov7/rbcdet.yaml', '/kaggle/working/yolov7/data/rbcdet.yaml')

In [ ]:
!sed -i 's/iou_t: 0.2/iou_t: 0.6/'  /kaggle/working/yolov7/data/hyp.scratch.p5.yaml

In [ ]:
!python train.py --batch 16 --epochs 25 --data /kaggle/working/yolov7/data/rbcdet.yaml --weights '/kaggle/working/yolov7/runs/train/exp3/weights/best.pt'   --img=640 --freeze 5 --hyp data/hyp.scratch.p5.yaml

In [ ]:
!python detect.py --weights /kaggle/working/yolov7/runs/train/exp4/weights/best.pt --img 256 --conf 0.40 --source /kaggle/input/testdetect/

In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

i = 0
limit = 10000 # max images to print
for imageName in glob.glob('/kaggle/working/yolov7/runs/detect/exp8/*.jpeg'):
    if i < limit:
      display(Image(filename=imageName))
      print("\n")
    i = i + 1

In [ ]:
import cv2
import torch
from models.experimental import attempt_load
from utils.general import check_img_size, non_max_suppression, scale_coords
from utils.plots import plot_one_box
from utils.torch_utils import select_device
# Set the device to run on

# Load the custom YOLOv7 model
model = attempt_load('/kaggle/input/bestdetect1/best (1).pt')
names = model.module.names if hasattr(model, 'module') else model.names
# Set the image size
imgsz = check_img_size(640, s=model.stride.max())

# Set the filename of the image
filename = '/kaggle/input/testdetect/image384.jpeg'

# Read the image
image = cv2.imread(filename)

# Check if the image was read correctly
if image is None:
    print('Error: Could not read image')
else:
    # Set the model's stride
    stride = int(model.stride.max())

    # Compute the new size of the image
    new_size = (image.shape[1] - image.shape[1] % stride, image.shape[0] - image.shape[0] % stride)

    # Resize the image
    image = cv2.resize(image, new_size)
    
    # Convert the image to a tensor
    img = torch.from_numpy(image.transpose((2, 0, 1)))
    
    # Reshape and normalize the image
    img = img.float()
    img /= 255.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)
    
    # Run the model on the image
    pred = model(img)[0]
    
    # Apply non-maximum suppression to the predictions
    pred = non_max_suppression(pred)
    
    # Process the predictions
    for det in pred:
        if len(det):
            # Rescale the coordinates to the original image size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], image.shape).round()
            
            # Draw the bounding boxes on the image
            for *xyxy, conf, cls in reversed(det):
                if conf > 0.4:
                    x1, y1, x2, y2=xyxy
                    label = f'{names[int(cls)]} {conf:.2f}'
                    face = image[int(y1):int(y2), int(x1):int(x2)]
                    print(face.shape)
                    _,face=cv2.imencode('.jpeg', face) 
                    display(Image(face))
            plot_one_box(xyxy, image, label=label)
            _,image=cv2.imencode('.jpeg', image) 
            display(Image(image))
                    
    